# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si; psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.847715525333                   -0.70    4.5    108ms
  2   -7.852385689960       -2.33       -1.53    1.0   17.7ms
  3   -7.852608526563       -3.65       -2.54    1.2   18.5ms
  4   -7.852645455493       -4.43       -2.78    2.5   23.2ms
  5   -7.852646116448       -6.18       -2.88    1.2   18.5ms
  6   -7.852646667205       -6.26       -3.89    1.0   18.0ms
  7   -7.852646685944       -7.73       -4.62    2.0   64.8ms
  8   -7.852646686684       -9.13       -5.11    1.5   19.6ms
  9   -7.852646686725      -10.39       -5.74    1.2   19.2ms
 10   -7.852646686729      -11.34       -5.98    2.0   22.4ms
 11   -7.852646686730      -12.41       -6.55    1.0   18.5ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.847776637273                   -0.70           4.5    418ms
  2   -7.852561107626       -2.32       -1.62   0.80    2.2    2.37s
  3   -7.852641777153       -4.09       -2.69   0.80    1.0    234ms
  4   -7.852646503862       -5.33       -3.43   0.80    1.8   19.8ms
  5   -7.852646681733       -6.75       -4.41   0.80    2.0   20.1ms
  6   -7.852646686633       -8.31       -4.88   0.80    2.5   22.5ms
  7   -7.852646686725      -10.04       -5.54   0.80    1.0   16.4ms
  8   -7.852646686730      -11.33       -6.48   0.80    1.5   19.3ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +1.242843227457                   -1.00    4.38s
  2   -1.332637854287        0.41       -0.69    108ms
  3   -3.494481637289        0.33       -0.46   33.0ms
  4   -4.449077121683       -0.02       -0.58   32.9ms
  5   -6.270454056228        0.26       -0.64   32.9ms
  6   -7.151745662839       -0.05       -0.97   33.0ms
  7   -7.477816097498       -0.49       -1.36   24.8ms
  8   -7.703511615218       -0.65       -1.70   24.8ms
  9   -7.767295068779       -1.20       -1.81   24.9ms
 10   -7.801719717790       -1.46       -1.99   24.8ms
 11   -7.827717495126       -1.59       -2.26   24.7ms
 12   -7.843291169873       -1.81       -2.11   24.8ms
 13   -7.848490781619       -2.28       -2.82   24.9ms
 14   -7.851570080660       -2.51       -2.92   25.1ms
 15   -7.852018004673       -3.35       -3.32   25.4ms
 16   -7.852419150247       -3.40       -3.69   25.0ms
 17   -7.85

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.847722213053                   -0.70    4.8   29.9ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645811909                   -1.63    13.9s
  2   -7.852646686730       -6.06       -3.69    3.62s
  3   -7.852646686730      -13.13       -7.17    120ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.861034723590568e-7
|ρ_newton - ρ_scfv| = 3.4228147312591993e-7
|ρ_newton - ρ_dm|   = 6.853243272703199e-10
